In [3]:


import matplotlib.pyplot as plt

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [5]:
(xtrain,ytrain),(xtest,ytest) = keras.datasets.mnist.load_data()
class preper():
    def __init__(self):
        return None
        
    def flatten(self,data):
        c=[]
        for i in data:
            c.append((i.reshape(1,784)/256)-0.5)
        return c
    def onehot(self,data):
        c=[]
        for i in data:
            a=np.zeros((1,10))
            a=a[0]
            a[i]=1
            c.append(a)
        return c

preper=preper()
xtrain=preper.flatten(xtrain)
ytrain=preper.onehot(ytrain)


In [6]:
class NN():
    def __init__(self,arc=[784,16,16,10]):
        self.W=[]
        self.arc=arc
        [self.W.append(np.random.random((self.arc[x],self.arc[x+1]))-0.5)for x in range(len(self.arc))[:-1]]
        self.B=[np.random.random((1,i))-0.5 for i in self.arc[1:]]
        print(self.B)#deletable
        self.probstack=[]#deletable
        self.Wstacktester=[] #deletable
        self.testerflag=True #deletable
        self.testerww=[] #deletable
        #print(self.W)
        #print(self.B)
        
    def FF(self,inp):
        self.O=[inp]#list of arrays
        self.Z=[inp]#list of arrays
        
        def relu(x):
            return np.maximum(0,x)
        def sigmoid(x):
            x=np.exp(x)
            s=np.sum(x)
            return x/s
        def da(w,b):
            #print(self.O[-1])
            self.Z.append(self.O[-1]@w+b)#append an array
            self.O.append(relu(self.Z[-1]))
            
        [da(w,b) for w,b in zip(self.W,self.B)]
        self.O[-1]= sigmoid(self.Z[-1])
        #print(self.B)#deletable
        return self.O[-1]
        
    def batchify(self,data):
        return [data[x:x+100] for x in range(len(data))[::100]]
    
    def renorm(self,a,b):
        c=[]
        for i,thing in enumerate(a):
            if not(isinstance(i,type(np.array([1]))) or isinstance(i,list)):
                c.append(thing+b[i])
            else:
                c.append(self.renorm(thing,b[i]))
        #self.c=c# deletable
        return c
    
    def renorm2(self,j,length):
        k=[]
        
        for i in j:
            if not(isinstance(i,type(np.array([1])))or isinstance(i,list)):
                k.append(i/length)
            else:
                k.append(self.renorm2(i,length))
        return k
    
    def destack(self,stack):
        j=stack[0]
        for i in stack[1:]:
            j=self.renorm(j,i)
        #return [np.array(i) for i in self.renorm2(j,len(stack)) ]
        return self.renorm2(j,len(stack))
    
    def BP(self,batch):
        deltaBstack=[]
        deltaWstack=[]
        if self.testerflag :
            self.testerflag=False
            self.testerww= self.B # why the fuck is this []?
        ###hoping the batch is list with items of form [(image,label)]..
        for I,i in batch:
            self.FF(I)
            dCdO=(self.O[-1]-i)*2
            dOdZ=np.diagflat(self.O[-1]*(1-self.O[-1]))
            dCdZ=dCdO@dOdZ
            #self.probstack.append((problem,I,i,self.O[-1]))#deletable
            def drelu(x):
                if x>0:
                    return 1
                else:
                    return 0
            drelu=np.vectorize(drelu)
            biastemplate=self.B.copy()
            weighttemplate=self.W.copy()

            for layer in range(len(self.arc))[::-1][:-1]:# 3,2,1 
                frm,to=weighttemplate[layer-1].shape
                weighttemplate[layer-1]=np.tile(dCdZ,(frm,1))*np.tile(self.O[layer-1].T,(1,to))
                biastemplate[layer-1]=dCdZ@np.eye(dCdZ.shape[1])
                if not(layer==1):
                    dCdO=dCdZ@self.W[layer-1].T
                    dCdZ=dCdO*drelu(self.Z[layer-1])
                
            deltaBstack.append(biastemplate) 
            deltaWstack.append(weighttemplate) 
            
        B=self.destack(deltaBstack)
        self.Wstacktester=deltaWstack
        W=self.destack(deltaWstack)
        alpha=-0.1
        B=[np.array(b)*alpha for b in B]
        #self.testerw.append(self.W)
        W=[np.array(w)*alpha for w in W]
        #self.testerww.append(self.W)
        self.B=self.renorm(self.B,B)
        self.W=self.renorm(self.W,W)
        
    def accuracy(self,batch):
        out=[(self.FF(I)-i)**2 for I,i in batch]
        return np.sum(out)
            

In [14]:
nn=NN()



[array([[-0.28138943, -0.48137174, -0.17860834,  0.47774382, -0.01266057,
         0.20910158, -0.42225397, -0.35678191,  0.09564004,  0.09139158,
         0.05126811, -0.12340529,  0.46810755, -0.39608702,  0.21750917,
         0.05542765]]), array([[-0.02948465,  0.15048114,  0.14960271, -0.025214  , -0.01570899,
        -0.10267969,  0.35368965,  0.10892471,  0.29331177, -0.17864567,
         0.15283427,  0.33982424,  0.03755565,  0.45646471,  0.05887733,
         0.07751781]]), array([[ 0.01346062, -0.4013631 ,  0.13135487, -0.32890035, -0.09969454,
         0.05764184, -0.05526691,  0.21565025,  0.03455624,  0.1862326 ]])]


                if(layer!=len(self.arc)-1):
                    biastemplate[layer-1]=drelu(self.Z[layer])*problem
                    step=np.tile(self.O[-1]*(1-self.O[-1]),(self.O[-1].shape[1],1))*np.eye(self.O[-1].shape[1])
                    step2=problem@step 
                else:
                    #problem@(dOl/dZl)@(dZl/dBl)
                    #self.eye=self.O[-1]
                    step=np.tile(self.O[-1]*(1-self.O[-1]),(self.O[-1].shape[1],1))*np.eye(self.O[-1].shape[1])
                    step2=problem@step 
                    biastemplate[layer-1]=step2@np.eye(self.O[-1].shape[1])
                    self.layers=[self.O[layer].shape for layer in range(4)]#deltable
                    
                self.testerlayer=self.O[layer].shape
                weighttemplate[layer-1]=np.tile(step2,(self.O[layer-1].shape[1],1))*np.tile(self.O[layer-1].T,(1,self.O[layer].shape[1]))

                problem=drelu(self.Z[layer]*problem)@self.W[layer-1].T

In [8]:
data=list(zip(xtrain,ytrain))
batched=nn.batchify(data) #[ [(image,label),(),(),()], [(image,label),(),(),()], .............] image is a 1*748 array

In [15]:
nn.accuracy(batched[0])

105.16879644561678

In [10]:
nn.FF(batched[0][0][0])

array([[1.69734333e-05, 2.75723861e-05, 2.57692448e-03, 1.52651019e-06,
        3.52987962e-03, 2.67247458e-02, 9.66713381e-01, 2.16677560e-04,
        4.17616767e-08, 1.92277069e-04]])

In [16]:

[nn.BP(batched[i]) for i in range(len(batched))]
print(5)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
nn.accuracy(batched[0])

42.981595889788714

In [31]:
nn.testerww

[array([[ 0.34062888, -0.47547913,  0.25699628, -0.13121289, -0.07783348,
          0.34131684, -0.14596772,  0.2224481 ,  0.3159467 , -0.08238976,
          0.27122278,  0.24014582, -0.40431421,  0.15036854,  0.45898189,
         -0.01967173]]),
 array([[-0.05126031,  0.17501898,  0.49401787, -0.27982131, -0.21293722,
          0.42663296,  0.08762134,  0.14802933,  0.06462902,  0.15573227,
          0.46846541, -0.45385251,  0.04688685, -0.01622721,  0.09620134,
          0.3255319 ]]),
 array([[-0.08916168, -0.31935458, -0.14253289, -0.0944817 , -0.37158285,
          0.10909869,  0.17375277,  0.24089925,  0.39472371, -0.18780483]])]

In [ ]:
# batched[0][0][0] is the first image

In [18]:
def indexofmax(i):
    m=max(i)
    for j,thing in enumerate(i):
        if m==thing:
            return j
def func0(I,i):
    x=nn.FF(I)[0]
    xx=indexofmax(x)
    y=indexofmax(i)
    return (xx,y)
    
print([func0(I,i) for I,i in batched[0]]) #predictions and real labels seem to match most of the times

[(5, 5), (0, 0), (1, 4), (1, 1), (9, 9), (4, 2), (1, 1), (3, 3), (1, 1), (4, 4), (3, 3), (1, 5), (3, 3), (6, 6), (1, 1), (9, 7), (2, 2), (8, 8), (6, 6), (7, 9), (4, 4), (0, 0), (4, 9), (1, 1), (6, 1), (3, 2), (9, 4), (3, 3), (7, 2), (9, 7), (8, 3), (3, 8), (6, 6), (9, 9), (0, 0), (5, 5), (6, 6), (0, 0), (9, 7), (6, 6), (1, 1), (5, 8), (7, 7), (9, 9), (1, 3), (9, 9), (9, 8), (3, 5), (5, 9), (3, 3), (3, 3), (0, 0), (7, 7), (8, 4), (7, 9), (8, 8), (0, 0), (9, 9), (4, 4), (1, 1), (4, 4), (4, 4), (6, 6), (0, 0), (4, 4), (3, 5), (6, 6), (1, 1), (3, 0), (0, 0), (1, 1), (7, 7), (1, 1), (6, 6), (3, 3), (3, 0), (3, 2), (1, 1), (4, 1), (7, 7), (0, 9), (0, 0), (2, 2), (6, 6), (7, 7), (3, 8), (7, 3), (9, 9), (0, 0), (9, 4), (6, 6), (7, 7), (4, 4), (6, 6), (8, 8), (0, 0), (7, 7), (8, 8), (3, 3), (1, 1)]


In [69]:
np.max(x)

0.45898189

In [1]:
[print((i,thing)) for i,thing in enumerate([1,2,3])]

(0, 1)
(1, 2)
(2, 3)


[None, None, None]

In [2]:
(lambda x:x**5)(9)

59049